## 背景

- **集体智慧**：为了创造新的想法，而将一群人的行为、偏好或思想组合在一起。
- **机器学习的局限**：机器学习算法受限于其在大量模式之上的归纳能力，且只能凭借已经见过的数据进行归纳。如果一个模式不同于算法先前所见的任何其他模式，那么它很有可能会被“误解”。
- **协作性过滤**：一个协作性过滤算法通常的做法是对一大群人进行搜索，并从中找出与我们品味相近的一群人。

## 搜集偏好

下面是一个涉及影评者及其对几部电影评分情况的字典：

In [1]:
from recommendations import critics
print(critics)
print("\nLisa Rose 对 Lady in the Water 的评分：",critics['Lisa Rose']['Lady in the Water'])
print("\nToby 评价过的电影及评分：", critics['Toby'])

{'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5, 'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5, 'The Night Listener': 3.0}, 'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0, 'You, Me and Dupree': 3.5}, 'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0, 'Superman Returns': 3.5, 'The Night Listener': 4.0}, 'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0, 'The Night Listener': 4.5, 'Superman Returns': 4.0, 'You, Me and Dupree': 2.5}, 'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0, 'You, Me and Dupree': 2.0}, 'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5}, 'Toby': {'Snakes on a Plane': 4.5, 'You, Me and Dupree': 1.0, 'Superman 

## 寻找相近的用户
搜集完人们的偏好数据之后，我们需要有一种方法来确定人们在品味方面的相似程度。为此，我们可以将每个人与所有其他人进行对比，并计算他们的**相似度评价值**。两套常用的计算相似度评价值的体系：**欧几里得距离**和**皮尔逊相关度**。

- **欧氏距离**：介绍略

In [2]:
# 通过欧氏距离计算两个人相似度评价的函数
from math import sqrt
def sim_distance(prefs, p1, p2):
    # 如果两人没有共同项，返回0
    sim = []
    for item in prefs[p1]:
        if item in prefs[p2]:
            sim.append(item)
            
    if len(sim) == 0:
        return 0
    
    # 计算欧氏距离
    distance = sqrt(sum([pow(prefs[p1][item]-prefs[p2][item], 2) for item in sim]))
    return 1/(1+distance) # 防止除数为0

In [3]:
sim_distance(critics, 'Lisa Rose', 'Gene Seymour')

0.29429805508554946

- **皮尔逊相关度**

该相关系数是判断两组数据与某一直线拟合程度的一种度量。它在数据不是很规范（normalized）的时候（比如，影评者对影片的评价总是相对于平均水平偏离很大时），会倾向于给出更好的结果。

其公式为：
$$P_{X,Y}=\cfrac{COV(X,Y)}{\sigma_x\sigma_y}
=\cfrac{E((X-\mu_X)(Y-\mu_Y))}{\sigma_x\sigma_y}
=\cfrac{E(XY)-E(X)E(Y)}{\sqrt{E(X^2)-E^2(X)}\sqrt{E(Y^2)-E^2(Y)}}$$

书中给出的公式为：
$$P_{X,Y}=\cfrac{\sum{XY}-\cfrac{\sum{X}\sum{Y}}{N}}{\sqrt{(\sum{X^2}-\cfrac{\sum^2{X}}{N})(\sum{Y^2}-\cfrac{\sum^2{Y}}{N})}}
=\cfrac{\cfrac{\sum{XY}-\cfrac{\sum{X}\sum{Y}}{N}}{N}}{\cfrac{\sqrt{(\sum{X^2}-\cfrac{\sum^2{X}}{N})(\sum{Y^2}-\cfrac{\sum^2{Y}}{N})}}{N}}
=\cfrac{\cfrac{\sum{XY}}{N}-\cfrac{\sum{X}}{N}\cfrac{\sum{Y}}{N}}{\sqrt{(\frac{\sum{X^2}}{N}-\frac{\sum^2{X}}{N^2})(\frac{\sum{Y^2}}{N}-\frac{\sum^2{Y}}{N^2})}}$$

两者是等价的，但书中给出的公式更便于计算

In [4]:
# 通过皮尔逊相关系数计算相似度
def sim_pearson(prefs, p1, p2):
    # 如果两人没有共同项，返回1
    sim = []
    for item in prefs[p1]:
        if item in prefs[p2]:
            sim.append(item)
    
    n = len(sim)
    if n == 0:
        return 1
    
    # 求和
    sum1 = sum([prefs[p1][item] for item in sim])
    sum2 = sum([prefs[p2][item] for item in sim])
    
    # 求平方和
    sum1_sq = sum([pow(prefs[p1][item], 2) for item in sim])
    sum2_sq = sum([pow(prefs[p2][item], 2) for item in sim])
    
    # 求乘积之和
    p_sum = sum([prefs[p1][item]*prefs[p2][item] for item in sim])
    
    # 计算相关系数
    num = p_sum - (sum1*sum2/n)
    den = sqrt((sum1_sq-pow(sum1,2)/n)*(sum2_sq-pow(sum2,2)/n))
    
    if den==0:
        return 0
    r = num/den
    return r

In [5]:
sim_pearson(critics, 'Lisa Rose', 'Gene Seymour')

0.39605901719066977

## 为评论者打分
既然已经有了对两个人进行比较的函数，下面可以编写函数，根据指定的人对每个人进行打分，并找出最接近的匹配结果。

In [6]:
def topMatches(prefs, person, n=5, sim_func=sim_pearson):
    '''
    perfs: 偏好字典
    n: 返回最接近的n个结果
    sim_func: 评价函数
    '''
    # 返回（相似度，名字）的元组
    scores = [(sim_func(prefs, person, other), other) for other in prefs if other != person] 
    scores.sort()
    scores.reverse()
    return scores[:n]

In [7]:
topMatches(critics, 'Toby', n=3)

[(0.9912407071619299, 'Lisa Rose'),
 (0.9244734516419049, 'Mick LaSalle'),
 (0.8934051474415647, 'Claudia Puig')]

## 推荐物品
我们大多数时候想要的不是找到一位趣味相投的影评人，而是一份影片的推荐。为此，我们需要通过一个经过加权的评价值为影片打分，影片最终的评分为每个影片人的评分在相似度上的加权平均。

In [8]:
def getRecommendations(prefs, person, sim_func=sim_pearson):
    res = {}
    sim_sum = {}
    for other in prefs:
        if other == person:
            continue
        sim = sim_func(prefs, person, other)
        
        if sim<=0:
            continue
        for item in prefs[other]:
            # 只对自己还未看过的影片评价
            if item not in prefs[person] or prefs[person][item] == 0:
                # 默认为0
                res.setdefault(item, 0)
                res[item] += sim*prefs[other][item]
                sim_sum.setdefault(item, 0)
                sim_sum[item] += sim
                
    rankings = [(v/sim_sum[k],k) for k,v in res.items()]
    
    rankings.sort()
    rankings.reverse()
    return rankings

In [9]:
getRecommendations(critics,'Toby')

[(3.3477895267131017, 'The Night Listener'),
 (2.8325499182641614, 'Lady in the Water'),
 (2.530980703765565, 'Just My Luck')]

In [10]:
getRecommendations(critics, 'Toby', sim_func=sim_distance) # 此处书上有误，distance没有取平方根

[(3.457128694491423, 'The Night Listener'),
 (2.778584003814924, 'Lady in the Water'),
 (2.422482042361917, 'Just My Luck')]

## 匹配商品
如果想了解哪些商品是彼此相近的，应该怎么做？我们可以通过查看那些人喜欢某一特定物品，以及这些人喜欢哪些其他物品来决定相似度。这和我们之前用来决定人和人之间相似度的方法是一样的，只需要将人和物品对换即可。

可能一下难以理解，下面简要说明：
- 相似度：根据**两人**对**不同影片**的评分计算 -> 根据**不同人**对**两部影片**的评分计算

- topMatch：找出最相似的人 -> 找出最相似的影片
- getRecommendations：根据**其他人与指定人相似度**的加权给指定人**没有看过的影片**打分 -> 根据**其他影片与指定影片相似度**的加权预测**没有看过这部影片的人**的评分

In [11]:
def transPrefs(prefs):
    res = {}
    for person in prefs:
        for item in prefs[person]:
            res.setdefault(item, {})
            res[item][person] = prefs[person][item]
    return res

In [12]:
print(transPrefs(critics))

{'Lady in the Water': {'Lisa Rose': 2.5, 'Gene Seymour': 3.0, 'Michael Phillips': 2.5, 'Mick LaSalle': 3.0, 'Jack Matthews': 3.0}, 'Snakes on a Plane': {'Lisa Rose': 3.5, 'Gene Seymour': 3.5, 'Michael Phillips': 3.0, 'Claudia Puig': 3.5, 'Mick LaSalle': 4.0, 'Jack Matthews': 4.0, 'Toby': 4.5}, 'Just My Luck': {'Lisa Rose': 3.0, 'Gene Seymour': 1.5, 'Claudia Puig': 3.0, 'Mick LaSalle': 2.0}, 'Superman Returns': {'Lisa Rose': 3.5, 'Gene Seymour': 5.0, 'Michael Phillips': 3.5, 'Claudia Puig': 4.0, 'Mick LaSalle': 3.0, 'Jack Matthews': 5.0, 'Toby': 4.0}, 'You, Me and Dupree': {'Lisa Rose': 2.5, 'Gene Seymour': 3.5, 'Claudia Puig': 2.5, 'Mick LaSalle': 2.0, 'Jack Matthews': 3.5, 'Toby': 1.0}, 'The Night Listener': {'Lisa Rose': 3.0, 'Gene Seymour': 3.0, 'Michael Phillips': 4.0, 'Claudia Puig': 4.5, 'Mick LaSalle': 3.0, 'Jack Matthews': 3.0}}


In [13]:
# 得到一组与《Superman Returns》最相近的影片
movies = transPrefs(critics)
topMatches(movies, 'Superman Returns')

[(0.6579516949597695, 'You, Me and Dupree'),
 (0.4879500364742689, 'Lady in the Water'),
 (0.11180339887498941, 'Snakes on a Plane'),
 (-0.1798471947990544, 'The Night Listener'),
 (-0.42289003161103106, 'Just My Luck')]

In [14]:
# 根据影片推荐人
getRecommendations(movies, 'Just My Luck')

[(4.0, 'Michael Phillips'), (3.0, 'Jack Matthews')]

In [15]:
# 上一个全是整数，比较有迷惑性
getRecommendations(movies, 'Lady in the Water')

[(3.610031066802182, 'Toby'), (3.4436241497684494, 'Claudia Puig')]